使用torch.nn包来构建神经网络。
nn包依赖autograd包来定义模型并求导。一个nn.Module包含各个层和一个forward(input)方法，该方法返回output 

In [1]:
#定义一个网络
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #1个输入图像通道，6个输出通道，5x5卷积核
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16 * 5 * 5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        #max pooling 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:] #除了批处理维度之外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
#net.parameters()返回可被学习的参数（权重）列表和值
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [3]:
#测试随机输入32x32
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.1657,  0.0314,  0.0473, -0.0544, -0.1273, -0.0486,  0.2184,  0.0561,
         -0.0803,  0.0407]], grad_fn=<AddmmBackward>)


In [4]:
#将所有参数的梯度缓存清零，然后进行随机梯度的反向传播
net.zero_grad()
out.backward(torch.randn(1,10))

回顾：
torch.Tensor 自动调用backward()实现支持自动梯度计算的多维数组
nn.Module:神经网络模块。封装参数、移动到GPU上运行、导出、加载等
nn.parameter:一种变量，当把它赋值给一个module时，被自动地注册为一个参数
autograd.function：实现一个自动求导操作的前向和反向定义
    
    

In [5]:
#nn包有许多不同的loss function，nn.MSELoss是一个比较简单的loss function
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(1.2446, grad_fn=<MseLossBackward>)


In [6]:
#现在，如果你反向追踪“loss”，使用它的“.grad_fn”属性，你会看到一个像这样的计算图：

#    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#          -> view -> linear -> relu -> linear -> relu -> linear
#          -> MSELoss
#          -> loss
#所以，当我们调用“loss.backward()”时，整个图是关于loss的求导，而图中所有具有“requires_grad=True”
#属性的Tensors，将会有它们的“.grad”Tensor来积累梯度。

In [7]:
#跟随一些反向的步骤：
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [8]:
#反向传播 Backprop
#为了反向传播误差，我们所要做的全部事情仅仅是一句“loss.backward()”。
#（1）首先需要清空现在的梯度，否则将来的梯度将被累积到现有的梯度中。
#（2）然后调用“loss.backward()”，并且在反向的前、后，看一看conv1的偏置梯度（conv1's bias gradients）
net.zero_grad() #清空所有参数的梯度缓冲区
print('conv1.bias.grad before backward') #反向传播之前的梯度
print(net.conv1.bias.grad) #conv1的偏导数

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0083, -0.0067,  0.0242, -0.0136,  0.0094,  0.0084])


In [9]:
#更新权重
#权重 = 权重 - 学习率 * 梯度       
# weight = weight - learning_rate * gradient
import torch.optim as optim
#创建优化器
optimizer = optim.SGD(net.parameters(),lr=0.01)

#训练中
optimizer.zero_grad()  #梯度缓冲区清零
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()    #更新梯度